In [ ]:
%matplotlib inline
import random, math, pylab, os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#padding on the y axis between label and axis
plt.rcParams['ytick.major.pad']='10'

dpi_out = 400
fig_width = 3.39
golden_mean = (np.sqrt(5)-1.0)/2.0    # Aesthetic ratio
fig_height = fig_width*golden_mean # height in inches
MAX_HEIGHT_INCHES = 8.0
if fig_height > MAX_HEIGHT_INCHES:
    print("WARNING: fig_height too large:" + fig_height + 
          "so will reduce to" + MAX_HEIGHT_INCHES + "inches.")
    fig_height = MAX_HEIGHT_INCHES
#fig, ax = plt.subplots(figsize = (fig_width, fig_height), dpi=400, frameon=True)
small_tick_size = 8
small_label_size = 8

# Tutorial 3: Clothes-pin model (hard disks in 1 dimension)

## From the naive to the efficient algorithm

### Naive approach

In the lecture, we discussed a naive algorithm for determining random legal configuration of clothes-pins on a line, obeying the following rule:

$$
\pi ( x_0, \ldots, x_{N-1}) =
\begin{cases}
  1 & \text{if configuration is legal} \\
  0 & \text{if configuration is illegal},
\end{cases}
$$


This respects the equiprobability principle: all configurations have the same energy. This system is the exact reduction of the hard disks system to one dimension. The program below implements the naive algorithm already presented. Note how each configuration is tested for overlaps.

In [ ]:
N = 15
L = 10.0
sigma = 0.1
n_configs = 100
for config in range(n_configs):
    n_trials = 0
    x = []
    while len(x) < N:
        x.append(random.uniform(sigma, L - sigma))
        for k in range(len(x) - 1):
            if abs(x[-1] - x[k]) < 2.0 * sigma:
                x = []
                n_trials += 1
                break
    print n_trials, " tabula rasa wipe outs before the following configuration was produced: "
    print x
    print

This algorithm is fine as it stands, insofar as it produces legal configurations with uniform probability, but it has a very high rejection rate, as was the case for its 2D equivalent, presented in the context of hard disks. The performance of the numerical study would be greatly enhanced if we were able to reduce this rejection rate. It is actually possible to devise an algorithm with a zero rejection rate, which still randomly samples the legal configurations with a uniform probability, as will be presented below. This is the first example of a rejection-free algorithm in this course.

### Clothes-pins on a line: rejection-free algorithm

The idea behind the algorithm is that in the space of all legal and illegal configuration, the subspace of illegal configuration can be identified, isolated, and removed from the configuration space which we sample. We will explain in class how to define a one-to-one correspondance between the coordinates $y_i$, representing the centers of the pins on a shorter line of length $L'= L - 2 N \sigma$, and the original coordinates $x_i$ of the pins on the line of length $L$"

$$
x_i = y_i + (2i + 1) \sigma
$$

It is crucial to note that not only is this correspondence  one-to-one between the two sets of coordinates, but the length of the dual washing line is always the same for a given value of $N$. As a consequence, any set of $N$ points on the short line may be inflated into a legal configuration of $N$ pins on the initial washing line of length $L$. This procedure is rejection-free, and is implemented in the program below. It is followed by the program which generates the histogram of the positions of the centers of the pins, as generated by the rejection-free algorithm. Its profile may be compared with the profile presernted in the lecture, generated with the naive algorithm. The outputs are identical, but the naive algorithm took a lot more time than the rejection-free version.

In [ ]:
N = 10
L = 20.0
sigma = 0.75
n_runs = 800

for run in range(n_runs):
    y = [random.uniform(0.0, L - 2 * N * sigma) for k in range(N)]
    y.sort()
    print [y[i] + (2 * i + 1) * sigma for i in range(N)]

In [ ]:
N = 10
L = 20.0
sigma = 0.75
n_runs = 1000000
data = []
for run in range(n_runs):
    y = [random.uniform(0.0, L - 2 * N * sigma) for k in range(N)]
    y.sort()
    data += [y[i] + (2 * i + 1) * sigma for i in range(N)]
pylab.xlabel('$x$', fontsize=14)
pylab.ylabel('$\pi(x)$', fontsize=14)
pylab.title('Density of %i clothes-pins ($\sigma$=%s) on a line of length L=%s' % (N, sigma, L))
pylab.hist(data, bins=200, normed=True)
pylab.savefig('plot-direct_pins_noreject.png')
pylab.show()

## Exact treatment of the problem

Related to the existence of an efficient rejection-free algorithm is the fact that it is possible to express analytically the partition funciton of the system, i.e. the number of legal configurations, for any given density $\eta$:

$$
Z(\eta) = \int\limits_\sigma^{L-\sigma}dx_0 \ldots dx_{N-1}\pi( x_0, \ldots, x_{N-1}),
$$
with $\pi(x_0,\ldots,x_{N-1}) =
\begin{cases}
  1 & \text{if configuration is legal} \\
  0 & \text{if configuration is illegal},
\end{cases}
$

In the previous algorithm, the sorting step was crucial. It can be mathematically formulated via the introduction of the function
$$\Theta(x_0, \ldots, x_{N-1})=
\begin{cases}
  1 & \text{if } x_0 < x_1 < \ldots < x_{N-1} \\
  0 & \text{otherwise},
\end{cases},$$which allows to pick the ordered permutation among the $N!$ possible ones, so that

$$
Z(\eta) = N! \int\limits_\sigma^{L-\sigma}dx_0 \ldots dx_{N-1}\pi( x_0, \ldots, x_{N-1}) \Theta(x_0, \ldots, x_{N-1}),
$$

At this point, the deflation step may be applied, which transforms the $x$ variables to the $y$ variables, and allows us to get rid of the overlap condition, leading to:
$$
Z(\eta) = N! \int\limits_0^{L-2N\sigma}dy_0 \ldots dy_{N-1} \Theta(y_0, \ldots, y_{N-1}) = \int\limits_0^{L-2N\sigma}dy_0 \ldots dy_{N-1} =
\begin{cases}
  (L-2N\sigma)^N & \text{if } L > 2N\sigma \\
  0 & \text{otherwise},
\end{cases} ,
$$

This result provides us with an analytical expression for the acceptance probability of the naive direct sampling algorithm presented above:
$$
p_{\text{acceptance}} = \dfrac{\text{number of legal configurations}}{\text{number of legal and illegal configurations}} = \dfrac{Z(\eta)}{Z(0)}
$$

### Questions

What is the acceptance probability of the naive algorithm given above?
1. $p_{\text{acc}} = \dfrac{(L-2N\sigma)^N}{(L-2\sigma)^N}$
2. $p_{\text{acc}} = \dfrac{(L-2\sigma)^N}{(L-2N\sigma)^N}$
3. $p_{\text{acc}} = \dfrac{(L-2N\sigma)^N}{L^N}$

Estimate the average number of rejected configurations per accepted configuration:
1. $1- \dfrac{(L-2N\sigma)^N}{(L-2\sigma)^N}$
2. $\dfrac{(L-2\sigma)^N}{(L-2N\sigma)^N}$
3. $\dfrac{(L-2N\sigma)^N}{L^N}$


Evaluate the average number of rejections before one legal configuration is generated, for density $\eta=0.75$ and $N=15$.
1. $\simeq 5 \times 10^6$
2. $\simeq 5 \times 10^7$
3. $\simeq 5 \times 10^8$

## Virial expansion revisited

It is possible to use the formula derived in the previous tutorial, $\dfrac{L}{N} \dfrac{\partial \ln(Z)}{\partial L}$, since we have an analytical expression for $Z$: $\dfrac{L}{N} \dfrac{\partial \ln(Z)}{\partial L} = \dfrac{L}{L-2N\sigma}= \dfrac{1}{1-\eta} = 1 + \eta + \eta^2 + \eta^3 + \mathcal{O}(\eta^4)$.

This series is convergent $\forall \eta < 1$. This means there is no phase transition. Conversely, $\eta=1$ appears as a special point where long range order is present.

The partition function grants us access to other quantities, such as $\pi(x)$, the probability of having a pin center at $x$, which exhibits the oscillatory behavior discussed in the lecture. Let us assume one pin is located at position $x$. The possibility place $k$ pins to is left, and $N-k-1$ pins to its right is given by
$$
Z_{k, x-\sigma} Z_{N-k-1, L-\sigma-x}=[(x-\sigma)-2\sigma k]^k [(L-\sigma-x)-2\sigma (N-k-1)]^{N-k-1}
$$

We can deduce that 

$$
\pi(x) = \sum\limits_{k=0}^{N-1}\dfrac{1}{Z_{N,L}}C_{N-1}^kZ_{k, x-\sigma} Z_{N-k-1, L-\sigma-x},
$$
where $C_{N-1}^k = \dfrac{(N-1)!}{(N-1-k)!k!}$

This formula is implemented in the program below.

In [ ]:
def binomialCoeff(n, k):
    result = 1
    for i in range(1, k+1):
        result = result * (n-i+1) / i
    return result

def Z(N, L, sigma):
    freespace = L - 2.0 * N * sigma
    if freespace > 0.0:
        result = freespace ** N
    else:
        result = 0.0
    return result

def pi(x, N, L, sigma):
    tot = 0.
    for k in range(0, N):
        Z1 = Z(k, x - sigma, sigma)
        Z2 = Z(N - k - 1, L - x - sigma, sigma)
        tot += binomialCoeff( N - 1, k) * Z1 * Z2
    Ztotal = Z(N, L, sigma)
    return tot / Ztotal

L = 20.0
N = 10
sigma = 0.75
xr = pylab.linspace(0.0, L, 1001)
yr = [pi(x, N, L, sigma) for x in xr]
pylab.plot(xr, yr, 'red', linewidth=1.0)
pylab.xlabel('$x$', fontsize=14)
pylab.ylabel('$\pi(x)$', fontsize=14)
pylab.title('Exact density of %i clothes-pins ($\sigma$=%s)\non a line of length L=%s' % (N, sigma, L))
pylab.savefig('plot-direct_pins_density.png')
pylab.show()

As was observed numerically, the function $\pi(x)$ increases steeply towards the poles, and is larger by a factor $\dfrac{1}{1-\eta}$ for $x=\sigma$ than for $x=L/2$: the pins become strongly attracted by the walls at high densities. We will now show that this is not a boundary effect, and that this attraction is also effective between pins.

## Liquid state, pair distribution, and the thermodynamic limit

### Periodic system setup

Up to this point, the pins system has been studied inside a box with walls, which cannot allow us to identify the nature of the peaks in the distribution close to the poles. They could be due to some boundary effect. We will now consider periodic boundary conditions, where such effect cannot be present. To this effect, the line will be replaced with a circle. A first pin is positioned at $x=L-\sigma$, which leaves $N-1$ pins to be placed, with $0 < x < L-2\sigma$. Note that this first pin simultaneously equips our periodic system with two poles, at $x=L-2\sigma$ and $x=0$. Therefore, the periodic system thus defined is perfectly equivalent to a linear system with $N-1$ pins and length $L-2\sigma$. In order to respect translational invariance, the configurations in the periodic system are shifted by a random amount, drawn uniformly between $0$ and $L$. This approach is implemented in the program below.

In [ ]:
N = 24
L = 30.0
sigma = 0.5
n_runs = 1000
data = []
for run in range(n_runs):
    Lprime = L - 2.0 * sigma
    y_sorted = [random.uniform(0, Lprime - 2.0 * (N - 1) * sigma) for k in range(N - 1)]
    y_sorted.sort()
    sample = [y_sorted[k] + (2.0 * k + 1.0) * sigma for k in range(N - 1)] + [L - sigma]
    shift = random.uniform(0, L)
    data += [(y + shift) % L for y in sample]
pylab.title('Density of %i clothes-pins ($\sigma$=%s) on a line of length L=%s' % (N, sigma, L))
pylab.xlabel('$x$', fontsize=14)
pylab.ylabel('$\pi(x)$', fontsize=14)
pylab.title('Density profile $\pi(x)$ for N=%i, $\sigma$=%.2f, L=%.1f' % (N, sigma, L))
pylab.hist(data, bins=100, normed=True)
pylab.savefig('plot-pins_noreject_periodic-N%04i-L%.1f-density.png' % (N, L))
pylab.show()

By experimenting with this program, we can see that the resulting function $\pi(x)$ converges to a flat function as the number of runs increases. This is in sharp contrast with the previous results, where the probability was peaked near the poles, and markedly oscillating elsewhere. Nevertheless, this does **not** mean that there is no correlation between the positions of the pins.

### Pair correlation function

In order to study the correlation between the positions of the pins, the profile of $\pi(x)$ is not enough. We need to introduce the pair correlation function $\pi(x, x')$, which represents the probability of having a pin at position $x'$, and a pin at position $x$. It is a good tool to study correlation, because it is trivially constant if there is no correlation between pin positions. The program below allows us to directly simulate and examine this pair correlation function.

In [ ]:
def dist(x1, x2, L):
    d_x = abs(x1 - x2) 
    return min(d_x, L - d_x)


L = 500.0
sigma = 0.5
density = 0.9
N = int(density * L / (2.0 * sigma))
n_runs = 100
x_max = 30.0  # maximum of the histogram range
data, pair_corr = [], []
for run in range(n_runs):
    Lprime = L - 2.0 * sigma
    y_sorted = [random.uniform(0, Lprime - 2.0 * (N - 1.0) * sigma) for k in xrange(N - 1)]
    y_sorted.sort()
    sample = [y_sorted[k] + (2.0 * k + 1.0) * sigma for k in xrange(N - 1) ] + [L - sigma]
    pair_corr += [dist(sample[i], sample[j], L) for i in xrange(N) for j in xrange(i)]

histo, bins, patches = pylab.hist(pair_corr, bins=800, normed=True)
pylab.xlim(0.0, x_max)
pylab.title('Pair-correlation function $\pi(x,y)$\nN=%i, $\sigma$=%.2f, L=%.1f, density=%.2f' % (N, sigma, L, density))
pylab.xlabel('$|x-y|$', fontsize=14)
pylab.ylabel('$\pi(|x-y|)$', fontsize=14)
pylab.savefig('plot-pins_noreject_periodic-N%04i-L%.1f-pair_corr.png' % (N, L))
pylab.show()
pylab.clf()
asymptotic_val = 1.0 / (L / 2.0)   # asymptotic value of the pair correlation function
pylab.semilogy(bins[:-1], [abs(y - asymptotic_val) for y in histo])
pylab.xlim(0.0, x_max)
pylab.title('Deviation of $\pi(x,y)$ from its asymptotic value\nN=%i, $\sigma$=%.2f, L=%.1f, density=%.2f' % (N, sigma, L, density))
pylab.xlabel('$|x-y|$', fontsize=14)
pylab.ylabel('$|\pi(|x-y|)-\pi_\mathrm{asympt}|$', fontsize=14)
pylab.savefig('plot-pins_noreject_periodic-N%04i-L%.1f-pair_corr_deviation.png' % (N, L))
pylab.show()

The pair-correlation function quickly converges to its thermodynamic limit, and oscillates in the same way as was observed for the distribution along a finite line. In fact, the pair correlation function has the same fundamental property in both cases, because the pin at $x$ acts as a pair of poles for the pin at position $x'$, as we discussed earlier. This observation allows us to show that

$$
\pi^{\text{periodic}}_{N, L}(x, x') = \pi^{\text{line}}_{N-1, L-2\sigma}(|x-x'| - \sigma)
$$

The analytical expression derived above for $Z$ and $\pi$ allow us conclude that this system is exactly solvable even in its periodic version.

### Physical interpretation

The pair-correlation function for this system oscillates inside a decaying envelope, and converges towards a constant value for large values of $|x-x'|$. The figure shown below the pair-correlation function illustrates this decay on a logarithmic scale. The close-to-linear behavior confirms that the oscillations decay exponentially: $\pi(x, x') - K \propto exp(-\dfrac{|x-x'|}{\xi})$. $\xi$ represents the correlation length between pins. for $\eta=0.8$, we have $\xi \simeq 2.4$, while for $\eta=0.9$, $\xi \simeq 6.4$, and $\xi$ exhibits a dramatic increase for $\eta = 0.95$: $\xi \simeq 26.2$. At distances larger than $\xi$, the system is homogeneous. The existence of such a correlation distance is the very definition of a liquid phase.

It can actually be shown that the correlation length diverges as the density goes to 1. This corresponds to the fact that the partition function is analytic for all values of $\eta$, but not for $\eta=1$, and that the only "phase transition" in this system happens when $L=2N\sigma$, with a close-packed system. This is a general feature of one dimensional systems with local interactions.